In [1]:
!pip install transformers torch sentence-transformers faiss-cpu wikipedia datetime pytz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 878.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.7 MB/s eta 0:00:00
  

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipedia
from datetime import datetime
import pytz
import random
import re
import warnings
warnings.filterwarnings('ignore')

class NovaAI:
    def __init__(self):
        # Model Initialization (Optimized for speed)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_name = "microsoft/DialoGPT-large"  # More powerful than medium
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name).to(self.device)
        self.generator = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device == "cuda" else -1,
        )

        # Knowledge Retrieval (RAG - Retrieval Augmented Generation)
        self.retriever = SentenceTransformer('all-MiniLM-L6-v2').to(self.device)
        self.knowledge_index = faiss.IndexFlatL2(384)  # Vector dimension for MiniLM

        # Conversation Memory
        self.chat_history = []
        self.max_history = 6  # Last 3 exchanges

        # Bot Personality & Config
        self.bot_name = "NovaAI"
        self.user_name = "User"
        self.timezone = pytz.timezone('UTC')  # Can be changed

        # Predefined Responses & Skills
        self.skills = {
            "time": self.get_current_time,
            "date": self.get_current_date,
            "weather": self.get_weather_response,
            "joke": self.tell_joke,
            "calculate": self.simple_math,
            "wiki": self.get_wikipedia_summary,
        }

        # Welcome Message
        self.welcome_msg = f"""
        🌟 **Welcome to {self.bot_name} - Your Intelligent AI Assistant!** 🌟
        - Ask me anything (knowledge, jokes, weather, time, etc.)
        - Type `/help` for commands
        - Type `quit` to exit
        Current Time: {self.get_current_time()}
        """

    # ==== Knowledge & Utility Functions ====
    def get_current_time(self):
        return datetime.now(self.timezone).strftime("%H:%M:%S")

    def get_current_date(self):
        return datetime.now(self.timezone).strftime("%Y-%m-%d")

    def get_weather_response(self):
        responses = [
            "I don’t have real-time weather, but you can check apps like AccuWeather!",
            "In my digital world, it’s always 72°F and sunny! ☀️",
            "I’m not a weather bot, but I hope it’s nice where you are!"
        ]
        return random.choice(responses)

    def tell_joke(self):
        jokes = [
            "Why don’t scientists trust atoms? Because they make up everything!",
            "Why did the scarecrow win an award? Because he was outstanding in his field!",
            "What’s orange and sounds like a parrot? A carrot!"
        ]
        return random.choice(jokes)

    def simple_math(self, query):
        try:
            expr = re.sub(r'[^0-9+\-*/(). ]', '', query)
            return f"Result: {eval(expr)}"  # Caution: Basic eval usage
        except:
            return "Sorry, I couldn’t compute that. Try something like '5 + 3'."

    def get_wikipedia_summary(self, query):
        try:
            summary = wikipedia.summary(query, sentences=2)
            return f"📚 Wikipedia: {summary}"
        except:
            return "Sorry, I couldn’t find a Wikipedia page for that."

    # ==== Response Generation Logic ====
    def retrieve_knowledge(self, query):
        """Retrieve relevant facts if available"""
        if "time" in query.lower():
            return f"⏰ Current time is {self.get_current_time()}"
        elif "date" in query.lower():
            return f"📅 Today is {self.get_current_date()}"
        elif "weather" in query.lower():
            return self.get_weather_response()
        elif "joke" in query.lower():
            return self.tell_joke()
        elif any(op in query for op in ['+', '-', '*', '/']):
            return self.simple_math(query)
        elif "who is" in query.lower() or "what is" in query.lower():
            return self.get_wikipedia_summary(query.split("is")[-1].strip())
        return None

    def generate_response(self, user_input):
        """Generate AI response with context"""
        # Check for commands
        if user_input.lower() in ['quit', 'exit', 'bye']:
            return "✨ Goodbye! Feel free to chat again later!"

        # Check for help command
        if user_input.lower() == '/help':
            return (
                "🛠️ **Commands:**\n"
                "- Ask me anything (general knowledge, jokes, etc.)\n"
                "- `time` → Get current time\n"
                "- `date` → Get today’s date\n"
                "- `weather` → Get weather response\n"
                "- `joke` → Hear a joke\n"
                "- `calculate <expression>` → Do math (e.g., `5 + 3`)\n"
                "- `who is <person>` → Wikipedia lookup\n"
            )

        # Retrieve knowledge if applicable
        knowledge_response = self.retrieve_knowledge(user_input)
        if knowledge_response:
            return knowledge_response

        # Generate AI response with conversation history
        prompt = (
            f"{' '.join(self.chat_history[-self.max_history:])}\n"
            f"User: {user_input}\n"
            f"AI:"
        )

        response = self.generator(
            prompt,
            max_length=150,
            num_return_sequences=1,
            temperature=0.8,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
        )

        # Extract and clean response
        ai_response = response[0]['generated_text'].split("AI:")[-1].strip()
        ai_response = re.sub(r'<\|.*?\|>', '', ai_response)  # Remove special tokens

        # Ensure response isn't just repeating input
        if ai_response.lower() == user_input.lower():
            return "🤔 Interesting! Could you elaborate?"

        return ai_response

    # ==== Main Chat Loop ====
    def start_chat(self):
        print(self.welcome_msg)

        while True:
            try:
                user_input = input(f"{self.user_name}: ").strip()
                if not user_input:
                    continue

                # Get response
                response = self.generate_response(user_input)
                print(f"{self.bot_name}: {response}")

                # Update chat history
                self.chat_history.extend([user_input, response])
                if len(self.chat_history) > self.max_history * 2:
                    self.chat_history = self.chat_history[-self.max_history * 2:]

                # Exit condition
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    break

            except KeyboardInterrupt:
                print(f"\n{self.bot_name}: ✨ See you next time!")
                break
            except Exception as e:
                print(f"{self.bot_name}: Oops! Something went wrong. Let's try again.")
                continue

In [ ]:
# Initialize & run
nova = NovaAI()
nova.start_chat()